In [137]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import glob
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re
from datetime import datetime, timedelta

In [138]:
# set working directory for accessing and saving files
os.chdir('d:\\Faith and Religion Stuff\\Come, Follow Me\\come-follow-me-breakdown-builder')

So before I can begin adapting my conference calculator into a Come, Follow Me Calculator, I need to scrape and adjust the starting and ending dates from the Come, Follow Me page. 

The initial link I'm starting with is the link to the 2024 Come, Follow Me landing page. 
* link: https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024?lang=eng

Having inspected the html elements, I have found that there are a few relevant elements, at least on the landing page:
* `<div class="sc-42v4-0 caNosj heading-Vx_DR heading-Vx_DR label">`: these class objects on the landing page mark the month divisions. 
* `<li data-date-end="2024-01-07" data-date-start="2024-01-01">`: these lines mark the start and end dates for each reading
* `<a href="/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/01?lang=eng"`: these are the links to each of the dedicated webpages for each week's readings and lessons. 
* `<p class="primaryMeta">January 1–7</p>`: these p class elements list the start and end dates in more readable format
* `<div class="sc-omeqik-7 iwWCCo">` : this div element marks the location in the html code of the "title" of the reading assignment linked above
    * `<h4 class="sc-12mz36o-0 jSCFto sc-omeqik-9 dbmmCm">`: this is an additional marker for the reading assignment title
    * `<p class="title">Introductory Pages of the Book of Mormon</p></h4></div>`: this p class is the title of the weekly reading assignment

All of these will be useful as I begin this process. 

I'll start, though, by just getting a response and confirming that my driver is up and running. 

In [139]:
cfm_link = 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024?lang=eng'
response = requests.get(cfm_link)
response

<Response [200]>

In [140]:
soup = BeautifulSoup(response.content, "html.parser")
content = soup.prettify()  # This formats the HTML in a (more) readable way
print(content)

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title data-react-helmet="true">
   Book of Mormon 2024
  </title>
  <meta content="4.22.0" name="version"/>
  <meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/>
  <meta content="" data-react-helmet="true" name="Search.doc-aid">
   <meta content="Book of Mormon 2024" data-react-helmet="true" name="title">
    <meta content="This manual contains lesson ideas and study materials for use at home and at church." data-react-helmet="true" name="description">
     <meta content="Book of Mormon 2024" data-react-helmet="true" property="og:title"/>
     <meta content="website" data-react-helmet="true" property="og:type"/>
     <meta content="https://www.churchofjesuschrist.org/study/eng/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/_manifest" data-react-helmet="true" property="og:url"/>
     <link href="https://ips-cdn-edge.churchofjesuschrist.org" rel="preconnect"/>
     <lin

A 200 response indicates that I can use requests and a driver to get information from the linked page. Now to set up my driver. 

In [141]:
chrome_driver_dir = r'D:\\Faith and Religion Stuff\\Come, Follow Me\\chromedriver-win64'
chrome_driver_path = os.path.join(chrome_driver_dir, 'chromedriver.exe')



After setting up the driver, I used it below to navigate to the landing page for the 2024 Book of Mormon Come, Follow Me assignments (the link above). From that page, I used webscraping to identify the line elements needed, and then extracted the start and end dates and saved them into respective lists.

In [142]:
cfm_links_list = []
cfm_links_list.append(input("Please paste the link to last year's Come, Follow Me landing page."))
cfm_links_list.append(input("Please paste the link to this year's Come, Follow Me landing page."))

In [143]:
service = Service(chrome_driver_path)

driver = webdriver.Chrome(service=service)
start_dates = []
end_dates = []
assignments = []
links = []

for link in cfm_links_list:
    driver.get(link)

    # Find all li elements with data-date-start and data-date-end attributes
    li_elements = driver.find_elements(By.CSS_SELECTOR, "li[data-date-start][data-date-end]")

    # Extract the data-date-start and data-date-end attributes
    for li in li_elements:
        start_dates.append(li.get_attribute("data-date-start"))
        end_dates.append(li.get_attribute("data-date-end"))
        assignments.append(li.find_element(By.CSS_SELECTOR,'p.title').text)
        a_element = li.find_element(By.TAG_NAME, 'a')
        links.append(a_element.get_attribute('href'))

# close the driver
driver.quit()

In [144]:
links

['https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/01?lang=eng',
 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/02?lang=eng',
 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/03?lang=eng',
 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/04?lang=eng',
 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/05?lang=eng',
 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/06?lang=eng',
 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/07?lang=eng',
 'https://www.churchofjesuschrist.org/study/manual/come-follow-me-for-home-and-church-book-of-mormon-2024/08?lang=eng',
 'https://www.churchofjesuschrist.org/st

I then copied the end dates list as 'sunday_dates' because I plan to finish the reading the day before the Sunday on which we'll be discussing the reading assignment in Sunday School. This assures that even if I don't have the chance to study scriptures before Church on Sunday, I'll still come to the Sunday School lesson having read everything we'll be discussing. 

In [145]:
sunday_dates = end_dates.copy()

I then create a dataframe 'readings_df' using the lists that I created. 

In [146]:
readings_df = pd.DataFrame({
    'reading_for_sunday': sunday_dates,
    'assignment': assignments,
    'wk_start_date': start_dates,
    'wk_end_date': end_dates,
    'link': links
})

readings_df

,reading_for_sunday,assignment,wk_start_date,wk_end_date,link
0,2024-01-07,Introductory Pages of the Book of Mormon,2024-01-01,2024-01-07,https://www.churchofjesuschrist.org/study/manu...
1,2024-01-14,1 Nephi 1–5,2024-01-08,2024-01-14,https://www.churchofjesuschrist.org/study/manu...
2,2024-01-21,1 Nephi 6–10,2024-01-15,2024-01-21,https://www.churchofjesuschrist.org/study/manu...
3,2024-01-28,1 Nephi 11–15,2024-01-22,2024-01-28,https://www.churchofjesuschrist.org/study/manu...
4,2024-02-04,1 Nephi 16–22,2024-01-29,2024-02-04,https://www.churchofjesuschrist.org/study/manu...
...,...,...,...,...,...
99,2025-11-30,Doctrine and Covenants 135–136,2025-11-24,2025-11-30,https://www.churchofjesuschrist.org/study/manu...
100,2025-12-07,Doctrine and Covenants 137–138,2025-12-01,2025-12-07,https://www.churchofjesuschrist.org/study/manu...
101,2025-12-14,The Articles of Faith and Official Declaration...,2025-12-08,2025-12-14,https://www.churchofjesuschrist.org/study/manu...
102,2025-12-21,The Family: A Proclamation to the World,2025-12-15,2025-12-21,https://www.churchofjesuschrist.org/study/manu...


Then I converted each of the columns from string objects to datetime objects. Having these stored and saved as datetime objects will allow me to extract things like the month and date and more easily. 

In [147]:
readings_df['wk_start_date'] = pd.to_datetime(readings_df.wk_start_date, format='%Y-%m-%d')
readings_df['wk_end_date'] = pd.to_datetime(readings_df.wk_end_date,format='%Y-%m-%d')
readings_df['reading_for_sunday'] = pd.to_datetime(readings_df.reading_for_sunday, format='%Y-%m-%d')

readings_df.dtypes

reading_for_sunday    datetime64[ns]
assignment                    object
wk_start_date         datetime64[ns]
wk_end_date           datetime64[ns]
link                          object
dtype: object

Then I subtracted one day from all of the dates in the 'start_date' and 'end_date' columns. Doing this sets my dates up in a way that when I use them in my calculator, I'll be able to assure that I finish reading assignments the day before they are discussed in Sunday School. 

I leave the 'reading_for_sunday' column the same, though, so I can pull valuable information about it below.

In [148]:
readings_df['wk_start_date'] = readings_df['wk_start_date'] - timedelta(days=1)
readings_df['wk_end_date'] = readings_df['wk_end_date'] - timedelta(days=1)

readings_df

,reading_for_sunday,assignment,wk_start_date,wk_end_date,link
0,2024-01-07,Introductory Pages of the Book of Mormon,2023-12-31,2024-01-06,https://www.churchofjesuschrist.org/study/manu...
1,2024-01-14,1 Nephi 1–5,2024-01-07,2024-01-13,https://www.churchofjesuschrist.org/study/manu...
2,2024-01-21,1 Nephi 6–10,2024-01-14,2024-01-20,https://www.churchofjesuschrist.org/study/manu...
3,2024-01-28,1 Nephi 11–15,2024-01-21,2024-01-27,https://www.churchofjesuschrist.org/study/manu...
4,2024-02-04,1 Nephi 16–22,2024-01-28,2024-02-03,https://www.churchofjesuschrist.org/study/manu...
...,...,...,...,...,...
99,2025-11-30,Doctrine and Covenants 135–136,2025-11-23,2025-11-29,https://www.churchofjesuschrist.org/study/manu...
100,2025-12-07,Doctrine and Covenants 137–138,2025-11-30,2025-12-06,https://www.churchofjesuschrist.org/study/manu...
101,2025-12-14,The Articles of Faith and Official Declaration...,2025-12-07,2025-12-13,https://www.churchofjesuschrist.org/study/manu...
102,2025-12-21,The Family: A Proclamation to the World,2025-12-14,2025-12-20,https://www.churchofjesuschrist.org/study/manu...


I then create two new columns, 'sunday_number' and 'sunday_class'. I fill the 'sunday_number' column simply with what number of Sunday it is for the month. The reason for this becomes clearer when I use that number to fill the 'sunday_class' column. Sunday School takes place on the first and third Sunday of every month (aside from General Conference months), and other meetings occur on the second and fourth Sundays. Fifth Sundays are generally left to be used as directed by Stake and Ward leadership. The code below fills the 'sunday_class' column accordingly. 

In [149]:
readings_df['sunday_number'] = pd.NA
readings_df['sunday_class'] = pd.NA

for item, row in readings_df.iterrows():
    day = row['reading_for_sunday'].day  # Access the day directly
    if day <= 7:
        readings_df.loc[item, 'sunday_number'] = 1
    elif day <= 14:
        readings_df.loc[item, 'sunday_number'] = 2
    elif day <= 21:
        readings_df.loc[item, 'sunday_number'] = 3
    elif day <= 28:
        readings_df.loc[item, 'sunday_number'] = 4
    else:  # day > 28
        readings_df.loc[item, 'sunday_number'] = 5

for item, row in readings_df.iterrows():
    if row['sunday_number'] == 1 and (row['reading_for_sunday'].month == 4 or row['reading_for_sunday'].month == 10):
        readings_df.loc[item, 'sunday_class'] = 'General Conference'
    elif row['sunday_number'] == 1 or row['sunday_number'] == 3:
        readings_df.loc[item, 'sunday_class'] = 'Sunday School'
    elif row['sunday_number'] == 2 or row['sunday_number'] == 4:
        readings_df.loc[item, 'sunday_class'] = 'EQ/RS/YM/YW'
    elif row['sunday_number'] == 5:
        readings_df.loc[item, 'sunday_class'] = 'Fifth Sunday'      

The cell below displays the readings_df dataframe to show that all the functions I performed above worked. Each `start_date` is 8 days before the `reading_for_sunday` date. Each `end_date` is 7 days after the `start_date`. Each `sunday_number` properly displays which Sunday of the month a given Sunday is, and each `sunday_class` item is listed correctly. 

In [150]:
readings_df

,reading_for_sunday,assignment,wk_start_date,wk_end_date,link,sunday_number,sunday_class
0,2024-01-07,Introductory Pages of the Book of Mormon,2023-12-31,2024-01-06,https://www.churchofjesuschrist.org/study/manu...,1,Sunday School
1,2024-01-14,1 Nephi 1–5,2024-01-07,2024-01-13,https://www.churchofjesuschrist.org/study/manu...,2,EQ/RS/YM/YW
2,2024-01-21,1 Nephi 6–10,2024-01-14,2024-01-20,https://www.churchofjesuschrist.org/study/manu...,3,Sunday School
3,2024-01-28,1 Nephi 11–15,2024-01-21,2024-01-27,https://www.churchofjesuschrist.org/study/manu...,4,EQ/RS/YM/YW
4,2024-02-04,1 Nephi 16–22,2024-01-28,2024-02-03,https://www.churchofjesuschrist.org/study/manu...,1,Sunday School
...,...,...,...,...,...,...,...
99,2025-11-30,Doctrine and Covenants 135–136,2025-11-23,2025-11-29,https://www.churchofjesuschrist.org/study/manu...,5,Fifth Sunday
100,2025-12-07,Doctrine and Covenants 137–138,2025-11-30,2025-12-06,https://www.churchofjesuschrist.org/study/manu...,1,Sunday School
101,2025-12-14,The Articles of Faith and Official Declaration...,2025-12-07,2025-12-13,https://www.churchofjesuschrist.org/study/manu...,2,EQ/RS/YM/YW
102,2025-12-21,The Family: A Proclamation to the World,2025-12-14,2025-12-20,https://www.churchofjesuschrist.org/study/manu...,3,Sunday School


Something interesting to note is that in 2024, both General Conferences were preceded by a 5th Sunday. The result of this is that twice in the year 2024, 5 weeks worth of reading material got broken down into daily chunks.

### 23 Oct 2024

Now that I know I can get all of the dates with a fair amount of ease, I want to make sure I can also get the reading assignments into the same dataframe. 

To do this, I just went back into the code above and added a line to pull and store the `p class='title'` elements in a list called assignments, and I adjusted the set up of the dataframe. The cell below shows that I renamed and reorganized the dataframe, and that everything worked as I had hoped it would. 

In [151]:
readings_df.columns

Index(['reading_for_sunday', 'assignment', 'wk_start_date', 'wk_end_date',
       'link', 'sunday_number', 'sunday_class'],
      dtype='object')

In [152]:
columns = ['reading_for_sunday','sunday_number', 'sunday_class','assignment', 'wk_start_date', 'wk_end_date','link']
readings_df = readings_df[columns]

In [153]:
readings_df

,reading_for_sunday,sunday_number,sunday_class,assignment,wk_start_date,wk_end_date,link
0,2024-01-07,1,Sunday School,Introductory Pages of the Book of Mormon,2023-12-31,2024-01-06,https://www.churchofjesuschrist.org/study/manu...
1,2024-01-14,2,EQ/RS/YM/YW,1 Nephi 1–5,2024-01-07,2024-01-13,https://www.churchofjesuschrist.org/study/manu...
2,2024-01-21,3,Sunday School,1 Nephi 6–10,2024-01-14,2024-01-20,https://www.churchofjesuschrist.org/study/manu...
3,2024-01-28,4,EQ/RS/YM/YW,1 Nephi 11–15,2024-01-21,2024-01-27,https://www.churchofjesuschrist.org/study/manu...
4,2024-02-04,1,Sunday School,1 Nephi 16–22,2024-01-28,2024-02-03,https://www.churchofjesuschrist.org/study/manu...
...,...,...,...,...,...,...,...
99,2025-11-30,5,Fifth Sunday,Doctrine and Covenants 135–136,2025-11-23,2025-11-29,https://www.churchofjesuschrist.org/study/manu...
100,2025-12-07,1,Sunday School,Doctrine and Covenants 137–138,2025-11-30,2025-12-06,https://www.churchofjesuschrist.org/study/manu...
101,2025-12-14,2,EQ/RS/YM/YW,The Articles of Faith and Official Declaration...,2025-12-07,2025-12-13,https://www.churchofjesuschrist.org/study/manu...
102,2025-12-21,3,Sunday School,The Family: A Proclamation to the World,2025-12-14,2025-12-20,https://www.churchofjesuschrist.org/study/manu...


### 28 Oct 2024

I've been busy over the past few days working out different coding things, and didn't feel like annotating, but I'm going back now and doing that. The work I did spanned most weekdays between the 23rd and 28th of October. 

After ensuring that I could get all the information above, I needed to create a grouping for each reading. This grouping is based on the date of the Sunday School meeting in which the reading assignment will be discussed. I also put in a user input section to ask if the user wants to breakdown readings across holidays or not, so that it will create those groupings accordingly. 

In [154]:
grouping_response = input(f'Do you want to study other readings in addition to holiday reading assignments? Please type yes or no.\n'
                          f'If you choose yes, the weeks leading up to and following those holidays will be more equalized, but you will need to read more the week of the holiday.\n'
                          'If you select no, the weeks leading up to or following holiday weeks will be standalone weeks with little to no equalization across weeks, but you will have less to read the week of the holiday.')
grouping_response.strip().lower()
if grouping_response == 'no':
    readings_df['reading_group'] = pd.NA
    reading_group_ct = 1
    for item, row in readings_df.iterrows():
        if row['assignment'] == 'Easter' or row['assignment'] == 'Christmas':
            reading_group_ct += 1
            readings_df.loc[item, 'reading_group'] = reading_group_ct
            reading_group_ct += 1
        else:
            readings_df.loc[item, 'reading_group'] = reading_group_ct
        if row['sunday_class'] == 'Sunday School':  
            reading_group_ct += 1
elif grouping_response == 'yes':
    readings_df['reading_group'] = pd.NA
    reading_group_ct = 1
    for item, row in readings_df.iterrows():
        readings_df.loc[item, 'reading_group'] = reading_group_ct
        if row['sunday_class'] == 'Sunday School':  
            reading_group_ct += 1

readings_df

C:\Users\bfran\AppData\Local\Temp\ipykernel_23900\2636228985.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readings_df['reading_group'] = pd.NA


,reading_for_sunday,sunday_number,sunday_class,assignment,wk_start_date,wk_end_date,link,reading_group
0,2024-01-07,1,Sunday School,Introductory Pages of the Book of Mormon,2023-12-31,2024-01-06,https://www.churchofjesuschrist.org/study/manu...,1
1,2024-01-14,2,EQ/RS/YM/YW,1 Nephi 1–5,2024-01-07,2024-01-13,https://www.churchofjesuschrist.org/study/manu...,2
2,2024-01-21,3,Sunday School,1 Nephi 6–10,2024-01-14,2024-01-20,https://www.churchofjesuschrist.org/study/manu...,2
3,2024-01-28,4,EQ/RS/YM/YW,1 Nephi 11–15,2024-01-21,2024-01-27,https://www.churchofjesuschrist.org/study/manu...,3
4,2024-02-04,1,Sunday School,1 Nephi 16–22,2024-01-28,2024-02-03,https://www.churchofjesuschrist.org/study/manu...,3
...,...,...,...,...,...,...,...,...
99,2025-11-30,5,Fifth Sunday,Doctrine and Covenants 135–136,2025-11-23,2025-11-29,https://www.churchofjesuschrist.org/study/manu...,43
100,2025-12-07,1,Sunday School,Doctrine and Covenants 137–138,2025-11-30,2025-12-06,https://www.churchofjesuschrist.org/study/manu...,43
101,2025-12-14,2,EQ/RS/YM/YW,The Articles of Faith and Official Declaration...,2025-12-07,2025-12-13,https://www.churchofjesuschrist.org/study/manu...,44
102,2025-12-21,3,Sunday School,The Family: A Proclamation to the World,2025-12-14,2025-12-20,https://www.churchofjesuschrist.org/study/manu...,44


After creating those groups, I ask the user what year the schedule is being made for and remove all reading assignments that won't be discussed in that year. I made sure to include above a question collecting the link for the previous and the current Come, Follow Me landing pages so I could make sure that the last weeks of the previous year get included in the current calendar. My reasoning for this is that since Sunday School occurs on the first Sunday of the month, it usually follows that the reading for the last week of the month (and therefore for the year) will be discussed on the first Sunday of the next month (and therefore on the first Sunday of the next year). 

In [155]:
reading_year = input('What year is the schedule being made for?')

reading_year = pd.to_datetime(reading_year, format = '%Y')

reading_year_mask = readings_df['reading_for_sunday'].dt.year == reading_year.year

valid_readings = readings_df[reading_year_mask]['reading_group'].unique()

readings_df = readings_df[readings_df['reading_group'].isin(valid_readings)]

readings_df

,reading_for_sunday,sunday_number,sunday_class,assignment,wk_start_date,wk_end_date,link,reading_group
50,2024-12-22,4,EQ/RS/YM/YW,Moroni 10,2024-12-15,2024-12-21,https://www.churchofjesuschrist.org/study/manu...,23
51,2024-12-29,5,Fifth Sunday,Christmas,2024-12-22,2024-12-28,https://www.churchofjesuschrist.org/study/manu...,23
52,2025-01-05,1,Sunday School,The Restoration of the Fulness of the Gospel o...,2024-12-29,2025-01-04,https://www.churchofjesuschrist.org/study/manu...,23
53,2025-01-12,2,EQ/RS/YM/YW,Doctrine and Covenants 1,2025-01-05,2025-01-11,https://www.churchofjesuschrist.org/study/manu...,24
54,2025-01-19,3,Sunday School,Joseph Smith—History 1:1–26,2025-01-12,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,24
55,2025-01-26,4,EQ/RS/YM/YW,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-01-25,https://www.churchofjesuschrist.org/study/manu...,25
56,2025-02-02,1,Sunday School,Doctrine and Covenants 3–5,2025-01-26,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,25
57,2025-02-09,2,EQ/RS/YM/YW,Doctrine and Covenants 6–9,2025-02-02,2025-02-08,https://www.churchofjesuschrist.org/study/manu...,26
58,2025-02-16,3,Sunday School,Doctrine and Covenants 10–11,2025-02-09,2025-02-15,https://www.churchofjesuschrist.org/study/manu...,26
59,2025-02-23,4,EQ/RS/YM/YW,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-02-22,https://www.churchofjesuschrist.org/study/manu...,27


In [156]:
#### Added from below on a later date 
readings_df['reading_group'] = readings_df['reading_group'] - (readings_df['reading_group'].min()-1)

readings_df

C:\Users\bfran\AppData\Local\Temp\ipykernel_23900\3552167813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readings_df['reading_group'] = readings_df['reading_group'] - (readings_df['reading_group'].min()-1)


,reading_for_sunday,sunday_number,sunday_class,assignment,wk_start_date,wk_end_date,link,reading_group
50,2024-12-22,4,EQ/RS/YM/YW,Moroni 10,2024-12-15,2024-12-21,https://www.churchofjesuschrist.org/study/manu...,1
51,2024-12-29,5,Fifth Sunday,Christmas,2024-12-22,2024-12-28,https://www.churchofjesuschrist.org/study/manu...,1
52,2025-01-05,1,Sunday School,The Restoration of the Fulness of the Gospel o...,2024-12-29,2025-01-04,https://www.churchofjesuschrist.org/study/manu...,1
53,2025-01-12,2,EQ/RS/YM/YW,Doctrine and Covenants 1,2025-01-05,2025-01-11,https://www.churchofjesuschrist.org/study/manu...,2
54,2025-01-19,3,Sunday School,Joseph Smith—History 1:1–26,2025-01-12,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,2
55,2025-01-26,4,EQ/RS/YM/YW,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-01-25,https://www.churchofjesuschrist.org/study/manu...,3
56,2025-02-02,1,Sunday School,Doctrine and Covenants 3–5,2025-01-26,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,3
57,2025-02-09,2,EQ/RS/YM/YW,Doctrine and Covenants 6–9,2025-02-02,2025-02-08,https://www.churchofjesuschrist.org/study/manu...,4
58,2025-02-16,3,Sunday School,Doctrine and Covenants 10–11,2025-02-09,2025-02-15,https://www.churchofjesuschrist.org/study/manu...,4
59,2025-02-23,4,EQ/RS/YM/YW,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-02-22,https://www.churchofjesuschrist.org/study/manu...,5


In [157]:
readings_df.columns

Index(['reading_for_sunday', 'sunday_number', 'sunday_class', 'assignment',
       'wk_start_date', 'wk_end_date', 'link', 'reading_group'],
      dtype='object')

In [158]:
readings_df_copy = readings_df

In [159]:
readings_df_copy

,reading_for_sunday,sunday_number,sunday_class,assignment,wk_start_date,wk_end_date,link,reading_group
50,2024-12-22,4,EQ/RS/YM/YW,Moroni 10,2024-12-15,2024-12-21,https://www.churchofjesuschrist.org/study/manu...,1
51,2024-12-29,5,Fifth Sunday,Christmas,2024-12-22,2024-12-28,https://www.churchofjesuschrist.org/study/manu...,1
52,2025-01-05,1,Sunday School,The Restoration of the Fulness of the Gospel o...,2024-12-29,2025-01-04,https://www.churchofjesuschrist.org/study/manu...,1
53,2025-01-12,2,EQ/RS/YM/YW,Doctrine and Covenants 1,2025-01-05,2025-01-11,https://www.churchofjesuschrist.org/study/manu...,2
54,2025-01-19,3,Sunday School,Joseph Smith—History 1:1–26,2025-01-12,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,2
55,2025-01-26,4,EQ/RS/YM/YW,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-01-25,https://www.churchofjesuschrist.org/study/manu...,3
56,2025-02-02,1,Sunday School,Doctrine and Covenants 3–5,2025-01-26,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,3
57,2025-02-09,2,EQ/RS/YM/YW,Doctrine and Covenants 6–9,2025-02-02,2025-02-08,https://www.churchofjesuschrist.org/study/manu...,4
58,2025-02-16,3,Sunday School,Doctrine and Covenants 10–11,2025-02-09,2025-02-15,https://www.churchofjesuschrist.org/study/manu...,4
59,2025-02-23,4,EQ/RS/YM/YW,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-02-22,https://www.churchofjesuschrist.org/study/manu...,5


In [172]:
#### revert to backup, remove later
readings_df = readings_df_copy

In [173]:
#### Added from below on a later date
subsets_list = []
columns = ['']

for i in range(1, readings_df['reading_group'].max()+1):
    subset_df = readings_df[readings_df['reading_group'] == i]
    subset_df['group_start_date'] = subset_df['wk_start_date'].min()
    subset_df['group_end_date'] = subset_df['wk_end_date'].max()
    subset_df['reading_for_sunday'] = subset_df['group_end_date'] + timedelta(days=1)
    subset_df['wk_assignment'] = subset_df['assignment']
    subset_df['group_assignment'] = '; '.join(subset_df['assignment'])
    columns = ['reading_group','reading_for_sunday','group_assignment','group_start_date',
               'group_end_date','wk_assignment','wk_start_date', 'wk_end_date','link']
    subset_df = subset_df[columns]

    subset_df['links_list'] = '; '.join(subset_df['link'])
    subset_df.drop('link',axis=1,inplace=True)
    links_split = subset_df['links_list'].str.split(';',expand=True)
    links_split.columns = [f'assignmt_{x+1}_link' for x in range(links_split.shape[1])]
    subset_df = pd.concat([subset_df,links_split],axis=1)
    subset_df.drop('links_list',axis=1,inplace=True)

    subset_df['wk_start_date'] = subset_df['wk_start_date'].dt.strftime('%d %b %Y')
    subset_df['start_dates'] = '; '.join(subset_df['wk_start_date'])
    subset_df.drop('wk_start_date',axis=1,inplace=True)
    start_dates_split = subset_df['start_dates'].str.split(';',expand = True)
    start_dates_split.columns = [f'wk_{y+1}_start_date' for y in range(start_dates_split.shape[1])]
    subset_df = pd.concat([subset_df,start_dates_split],axis=1)
    subset_df.drop('start_dates',axis=1, inplace=True)
    
    subset_df['wk_end_date'] = subset_df['wk_end_date'].dt.strftime('%d %b %Y')
    subset_df['end_dates'] = '; '.join(subset_df['wk_end_date'])
    subset_df.drop('wk_end_date',axis=1,inplace=True)
    end_dates_split = subset_df['end_dates'].str.split(';',expand=True)
    end_dates_split.columns = [f'wk_{z+1}_end_date' for z in range(end_dates_split.shape[1])]
    subset_df = pd.concat([subset_df,end_dates_split],axis=1)
    subset_df.drop('end_dates',axis=1,inplace=True)
    
    subsets_list.append(subset_df)

readings_df = pd.concat(subsets_list, ignore_index=True)

readings_df

C:\Users\bfran\AppData\Local\Temp\ipykernel_23900\3482188190.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['group_start_date'] = subset_df['wk_start_date'].min()
C:\Users\bfran\AppData\Local\Temp\ipykernel_23900\3482188190.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['group_end_date'] = subset_df['wk_end_date'].max()
C:\Users\bfran\AppData\Local\Temp\ipykernel_23900\3482188190.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

,reading_group,reading_for_sunday,group_assignment,group_start_date,group_end_date,wk_assignment,assignmt_1_link,assignmt_2_link,assignmt_3_link,wk_1_start_date,...,wk_3_start_date,wk_1_end_date,wk_2_end_date,wk_3_end_date,assignmt_4_link,assignmt_5_link,wk_4_start_date,wk_5_start_date,wk_4_end_date,wk_5_end_date
0,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,Moroni 10,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,15 Dec 2024,...,29 Dec 2024,21 Dec 2024,28 Dec 2024,04 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,Christmas,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,15 Dec 2024,...,29 Dec 2024,21 Dec 2024,28 Dec 2024,04 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,The Restoration of the Fulness of the Gospel o...,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,15 Dec 2024,...,29 Dec 2024,21 Dec 2024,28 Dec 2024,04 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN
3,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,Doctrine and Covenants 1,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,05 Jan 2025,...,NaN,11 Jan 2025,18 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,Joseph Smith—History 1:1–26,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,05 Jan 2025,...,NaN,11 Jan 2025,18 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,Doctrine and Covenants 2; Joseph Smith—History...,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,19 Jan 2025,...,NaN,25 Jan 2025,01 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,Doctrine and Covenants 3–5,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,19 Jan 2025,...,NaN,25 Jan 2025,01 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,2025-02-16,Doctrine and Covenants 6–9; Doctrine and Coven...,2025-02-02,2025-02-15,Doctrine and Covenants 6–9,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,02 Feb 2025,...,NaN,08 Feb 2025,15 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,2025-02-16,Doctrine and Covenants 6–9; Doctrine and Coven...,2025-02-02,2025-02-15,Doctrine and Covenants 10–11,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,02 Feb 2025,...,NaN,08 Feb 2025,15 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5,2025-03-02,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-03-01,Doctrine and Covenants 12–17; Joseph Smith—His...,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,16 Feb 2025,...,NaN,22 Feb 2025,01 Mar 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I then remove the last reading assignment, as it'll be discussed in the following year and therefore should be included in the following year. 

In [174]:
last_reading_num = readings_df['reading_group'].max()

readings_df = readings_df[readings_df['reading_group']!= last_reading_num]

readings_df

,reading_group,reading_for_sunday,group_assignment,group_start_date,group_end_date,wk_assignment,assignmt_1_link,assignmt_2_link,assignmt_3_link,wk_1_start_date,...,wk_3_start_date,wk_1_end_date,wk_2_end_date,wk_3_end_date,assignmt_4_link,assignmt_5_link,wk_4_start_date,wk_5_start_date,wk_4_end_date,wk_5_end_date
0,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,Moroni 10,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,15 Dec 2024,...,29 Dec 2024,21 Dec 2024,28 Dec 2024,04 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,Christmas,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,15 Dec 2024,...,29 Dec 2024,21 Dec 2024,28 Dec 2024,04 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,The Restoration of the Fulness of the Gospel o...,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,15 Dec 2024,...,29 Dec 2024,21 Dec 2024,28 Dec 2024,04 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN
3,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,Doctrine and Covenants 1,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,05 Jan 2025,...,NaN,11 Jan 2025,18 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,Joseph Smith—History 1:1–26,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,05 Jan 2025,...,NaN,11 Jan 2025,18 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,Doctrine and Covenants 2; Joseph Smith—History...,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,19 Jan 2025,...,NaN,25 Jan 2025,01 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,Doctrine and Covenants 3–5,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,19 Jan 2025,...,NaN,25 Jan 2025,01 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,2025-02-16,Doctrine and Covenants 6–9; Doctrine and Coven...,2025-02-02,2025-02-15,Doctrine and Covenants 6–9,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,02 Feb 2025,...,NaN,08 Feb 2025,15 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,2025-02-16,Doctrine and Covenants 6–9; Doctrine and Coven...,2025-02-02,2025-02-15,Doctrine and Covenants 10–11,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,02 Feb 2025,...,NaN,08 Feb 2025,15 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5,2025-03-02,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-03-01,Doctrine and Covenants 12–17; Joseph Smith—His...,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,16 Feb 2025,...,NaN,22 Feb 2025,01 Mar 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The result of all of that is a weekly reading assignment for the year. However, some of those reading assignments contain multiple books. For example, when reading the New Testament, the reader will be assigned chapters and sections of Matthew, Mark, Luke, and John all at the same time. In order to account for that, I split each reading asssignment on the semi-colon, the typical marker for a different chapter or book, and put each chunk into different columns. This would put Mark 1, Luke 3, and John 2 (for example) all in their own columns. 

In [175]:
# Split the assignments column into multiple columns based on the semicolon
assignments_split = readings_df['group_assignment'].str.split(';', expand=True)

# Rename the new columns (optional)
assignments_split.columns = [f'assignment_{i+1}' for i in range(assignments_split.shape[1])]

# Concatenate the new columns back to the original DataFrame
readings_df = pd.concat([readings_df, assignments_split], axis=1)

readings_df.drop('wk_assignment',axis=1, inplace=True)

readings_df.drop_duplicates(inplace=True)

readings_df

,reading_group,reading_for_sunday,group_assignment,group_start_date,group_end_date,assignmt_1_link,assignmt_2_link,assignmt_3_link,wk_1_start_date,wk_2_start_date,...,assignmt_5_link,wk_4_start_date,wk_5_start_date,wk_4_end_date,wk_5_end_date,assignment_1,assignment_2,assignment_3,assignment_4,assignment_5
0,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,15 Dec 2024,22 Dec 2024,...,NaN,NaN,NaN,NaN,NaN,Moroni 10,Christmas,The Restoration of the Fulness of the Gospel ...,None,None
3,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,05 Jan 2025,12 Jan 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 1,Joseph Smith—History 1:1–26,None,None,None
5,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,19 Jan 2025,26 Jan 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 2,Joseph Smith—History 1:27–65,Doctrine and Covenants 3–5,None,None
7,4,2025-02-16,Doctrine and Covenants 6–9; Doctrine and Coven...,2025-02-02,2025-02-15,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,02 Feb 2025,09 Feb 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 6–9,Doctrine and Covenants 10–11,None,None,None
9,5,2025-03-02,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-03-01,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,16 Feb 2025,23 Feb 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 12–17,Joseph Smith—History 1:66–75,Doctrine and Covenants 18,None,None
11,6,2025-03-16,Doctrine and Covenants 19; Doctrine and Covena...,2025-03-02,2025-03-15,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,02 Mar 2025,09 Mar 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 19,Doctrine and Covenants 20–22,None,None,None
13,7,2025-04-20,Doctrine and Covenants 23–26; Doctrine and Cov...,2025-03-16,2025-04-19,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,https://www.churchofjesuschrist.org/study/man...,16 Mar 2025,23 Mar 2025,...,https://www.churchofjesuschrist.org/study/man...,06 Apr 2025,13 Apr 2025,12 Apr 2025,19 Apr 2025,Doctrine and Covenants 23–26,Doctrine and Covenants 27–28,Doctrine and Covenants 29,Doctrine and Covenants 30–36,Easter
18,8,2025-05-04,Doctrine and Covenants 37–40; Doctrine and Cov...,2025-04-20,2025-05-03,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,20 Apr 2025,27 Apr 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 37–40,Doctrine and Covenants 41–44,None,None,None
20,9,2025-05-18,Doctrine and Covenants 45; Doctrine and Covena...,2025-05-04,2025-05-17,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,04 May 2025,11 May 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 45,Doctrine and Covenants 46–48,None,None,None
22,10,2025-06-01,Doctrine and Covenants 49–50; Doctrine and Cov...,2025-05-18,2025-05-31,https://www.churchofjesuschrist.org/study/manu...,https://www.churchofjesuschrist.org/study/man...,NaN,18 May 2025,25 May 2025,...,NaN,NaN,NaN,NaN,NaN,Doctrine and Covenants 49–50,Doctrine and Covenants 51–57,None,None,None


In [181]:
assignment_col_pattern = r'assignment_\d+'
wk_start_pattern = r'wk_\d+_start_date'
wk_end_pattern = r'wk_\d+_end_date'
link_col_pattern = r'assignmt_\d+_link'

reading_df_cols = readings_df.columns

assignment_cols = []
wk_start_cols = []
wk_end_cols = []
link_cols = []

for col in reading_df_cols:
    if re.match(assignment_col_pattern, col):
        assignment_cols.append(col)
    if re.match(wk_start_pattern, col):
        wk_start_cols.append(col)
    if re.match(wk_end_pattern, col):
        wk_end_cols.append(col)
    if re.match(link_col_pattern, col):
        link_cols.append(col)

as_and_ls = []
ss_and_es = []

for i in range(len(assignment_cols)):
    as_and_ls.append(assignment_cols[i])
    as_and_ls.append(link_cols[i])

for x in range(len(wk_start_cols)):
    ss_and_es.append(wk_start_cols[x])
    ss_and_es.append(wk_end_cols[x])

as_and_ls

['assignment_1',
 'assignmt_1_link',
 'assignment_2',
 'assignmt_2_link',
 'assignment_3',
 'assignmt_3_link',
 'assignment_4',
 'assignmt_4_link',
 'assignment_5',
 'assignmt_5_link']

In [182]:
readings_df.columns

Index(['reading_group', 'reading_for_sunday', 'group_assignment',
       'group_start_date', 'group_end_date', 'assignmt_1_link',
       'assignmt_2_link', 'assignmt_3_link', 'wk_1_start_date',
       'wk_2_start_date', 'wk_3_start_date', 'wk_1_end_date', 'wk_2_end_date',
       'wk_3_end_date', 'assignmt_4_link', 'assignmt_5_link',
       'wk_4_start_date', 'wk_5_start_date', 'wk_4_end_date', 'wk_5_end_date',
       'assignment_1', 'assignment_2', 'assignment_3', 'assignment_4',
       'assignment_5'],
      dtype='object')

In [184]:
columns = ['reading_group', 'reading_for_sunday', 'group_assignment',
       'group_start_date', 'group_end_date']

for i in range(len(as_and_ls)):
    columns.append(as_and_ls[i])

for x in range(len(ss_and_es)):
    columns.append(ss_and_es[x])

readings_df = readings_df[columns]

In [185]:
readings_df

,reading_group,reading_for_sunday,group_assignment,group_start_date,group_end_date,assignment_1,assignmt_1_link,assignment_2,assignmt_2_link,assignment_3,...,wk_1_start_date,wk_1_end_date,wk_2_start_date,wk_2_end_date,wk_3_start_date,wk_3_end_date,wk_4_start_date,wk_4_end_date,wk_5_start_date,wk_5_end_date
0,1,2025-01-05,Moroni 10; Christmas; The Restoration of the F...,2024-12-15,2025-01-04,Moroni 10,https://www.churchofjesuschrist.org/study/manu...,Christmas,https://www.churchofjesuschrist.org/study/man...,The Restoration of the Fulness of the Gospel ...,...,15 Dec 2024,21 Dec 2024,22 Dec 2024,28 Dec 2024,29 Dec 2024,04 Jan 2025,NaN,NaN,NaN,NaN
3,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,Doctrine and Covenants 1,https://www.churchofjesuschrist.org/study/manu...,Joseph Smith—History 1:1–26,https://www.churchofjesuschrist.org/study/man...,None,...,05 Jan 2025,11 Jan 2025,12 Jan 2025,18 Jan 2025,NaN,NaN,NaN,NaN,NaN,NaN
5,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,Doctrine and Covenants 2,https://www.churchofjesuschrist.org/study/manu...,Joseph Smith—History 1:27–65,https://www.churchofjesuschrist.org/study/man...,Doctrine and Covenants 3–5,...,19 Jan 2025,25 Jan 2025,26 Jan 2025,01 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN
7,4,2025-02-16,Doctrine and Covenants 6–9; Doctrine and Coven...,2025-02-02,2025-02-15,Doctrine and Covenants 6–9,https://www.churchofjesuschrist.org/study/manu...,Doctrine and Covenants 10–11,https://www.churchofjesuschrist.org/study/man...,None,...,02 Feb 2025,08 Feb 2025,09 Feb 2025,15 Feb 2025,NaN,NaN,NaN,NaN,NaN,NaN
9,5,2025-03-02,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-03-01,Doctrine and Covenants 12–17,https://www.churchofjesuschrist.org/study/manu...,Joseph Smith—History 1:66–75,https://www.churchofjesuschrist.org/study/man...,Doctrine and Covenants 18,...,16 Feb 2025,22 Feb 2025,23 Feb 2025,01 Mar 2025,NaN,NaN,NaN,NaN,NaN,NaN
11,6,2025-03-16,Doctrine and Covenants 19; Doctrine and Covena...,2025-03-02,2025-03-15,Doctrine and Covenants 19,https://www.churchofjesuschrist.org/study/manu...,Doctrine and Covenants 20–22,https://www.churchofjesuschrist.org/study/man...,None,...,02 Mar 2025,08 Mar 2025,09 Mar 2025,15 Mar 2025,NaN,NaN,NaN,NaN,NaN,NaN
13,7,2025-04-20,Doctrine and Covenants 23–26; Doctrine and Cov...,2025-03-16,2025-04-19,Doctrine and Covenants 23–26,https://www.churchofjesuschrist.org/study/manu...,Doctrine and Covenants 27–28,https://www.churchofjesuschrist.org/study/man...,Doctrine and Covenants 29,...,16 Mar 2025,22 Mar 2025,23 Mar 2025,29 Mar 2025,30 Mar 2025,05 Apr 2025,06 Apr 2025,12 Apr 2025,13 Apr 2025,19 Apr 2025
18,8,2025-05-04,Doctrine and Covenants 37–40; Doctrine and Cov...,2025-04-20,2025-05-03,Doctrine and Covenants 37–40,https://www.churchofjesuschrist.org/study/manu...,Doctrine and Covenants 41–44,https://www.churchofjesuschrist.org/study/man...,None,...,20 Apr 2025,26 Apr 2025,27 Apr 2025,03 May 2025,NaN,NaN,NaN,NaN,NaN,NaN
20,9,2025-05-18,Doctrine and Covenants 45; Doctrine and Covena...,2025-05-04,2025-05-17,Doctrine and Covenants 45,https://www.churchofjesuschrist.org/study/manu...,Doctrine and Covenants 46–48,https://www.churchofjesuschrist.org/study/man...,None,...,04 May 2025,10 May 2025,11 May 2025,17 May 2025,NaN,NaN,NaN,NaN,NaN,NaN
22,10,2025-06-01,Doctrine and Covenants 49–50; Doctrine and Cov...,2025-05-18,2025-05-31,Doctrine and Covenants 49–50,https://www.churchofjesuschrist.org/study/manu...,Doctrine and Covenants 51–57,https://www.churchofjesuschrist.org/study/man...,None,...,18 May 2025,24 May 2025,25 May 2025,31 May 2025,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
chapter_pattern = r'\s\d+.*\d*'

for col in assignment_cols:
    readings_df[f'{col}_book'] = pd.NA  # New column for each book title
    readings_df[f'{col}_chapters'] = pd.NA # New column for each set of chapters in each part of the assignment
    for index, row in readings_df.iterrows():
        if pd.notna(row[col]):  # Ensure the cell has content
            match = re.search(chapter_pattern, row[col])
            if match:  # Check if match was found
                chapter_assignment = match.group(0).strip()
                book_assignment = row[col].replace(chapter_assignment,'').strip()

                readings_df.at[index, f'{col}_book'] = book_assignment
                readings_df.at[index, f'{col}_chapters'] = chapter_assignment
            else:
                readings_df.at[index, f'{col}_book'] = row[col]  # Keep the original value if no title match
                readings_df.at[index, f'{col}_chapters'] = pd.NA


In [118]:
readings_df

,reading_group,reading_for_sunday,group_assignment,group_start_date,group_end_date,wk_assignment,link_1,wk_1_start_date,wk_1_end_date,link_2,...,assignment_1_book,assignment_1_chapters,assignment_2_book,assignment_2_chapters,assignment_3_book,assignment_3_chapters,assignment_4_book,assignment_4_chapters,assignment_5_book,assignment_5_chapters
0,1,2025-01-05,The Restoration of the Fulness of the Gospel o...,2024-12-29,2025-01-04,The Restoration of the Fulness of the Gospel o...,https://www.churchofjesuschrist.org/study/manu...,2024-12-29,2025-01-04,NaN,...,The Restoration of the Fulness of the Gospel o...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,Doctrine and Covenants 1,https://www.churchofjesuschrist.org/study/manu...,2025-01-05,2025-01-11,https://www.churchofjesuschrist.org/study/manu...,...,Doctrine and Covenants,1,Joseph Smith—History,1:1–26,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2,2025-01-19,Doctrine and Covenants 1; Joseph Smith—History...,2025-01-05,2025-01-18,Joseph Smith—History 1:1–26,https://www.churchofjesuschrist.org/study/manu...,2025-01-12,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,...,Doctrine and Covenants,1,Joseph Smith—History,1:1–26,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,Doctrine and Covenants 2; Joseph Smith—History...,https://www.churchofjesuschrist.org/study/manu...,2025-01-19,2025-01-25,https://www.churchofjesuschrist.org/study/manu...,...,Doctrine and Covenants,2,Joseph Smith—History,1:27–65,Doctrine and Covenants,3–5,<NA>,<NA>,<NA>,<NA>
4,3,2025-02-02,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-02-01,Doctrine and Covenants 3–5,https://www.churchofjesuschrist.org/study/manu...,2025-01-26,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,...,Doctrine and Covenants,2,Joseph Smith—History,1:27–65,Doctrine and Covenants,3–5,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,43,2025-12-07,Doctrine and Covenants 133–134; Doctrine and C...,2025-11-16,2025-12-06,Doctrine and Covenants 133–134,https://www.churchofjesuschrist.org/study/manu...,2025-11-16,2025-11-22,https://www.churchofjesuschrist.org/study/manu...,...,Doctrine and Covenants,133–134,Doctrine and Covenants,135–136,Doctrine and Covenants,137–138,<NA>,<NA>,<NA>,<NA>
99,43,2025-12-07,Doctrine and Covenants 133–134; Doctrine and C...,2025-11-16,2025-12-06,Doctrine and Covenants 135–136,https://www.churchofjesuschrist.org/study/manu...,2025-11-23,2025-11-29,https://www.churchofjesuschrist.org/study/manu...,...,Doctrine and Covenants,133–134,Doctrine and Covenants,135–136,Doctrine and Covenants,137–138,<NA>,<NA>,<NA>,<NA>
100,43,2025-12-07,Doctrine and Covenants 133–134; Doctrine and C...,2025-11-16,2025-12-06,Doctrine and Covenants 137–138,https://www.churchofjesuschrist.org/study/manu...,2025-11-30,2025-12-06,https://www.churchofjesuschrist.org/study/manu...,...,Doctrine and Covenants,133–134,Doctrine and Covenants,135–136,Doctrine and Covenants,137–138,<NA>,<NA>,<NA>,<NA>
101,44,2025-12-21,The Articles of Faith and Official Declaration...,2025-12-07,2025-12-20,The Articles of Faith and Official Declaration...,https://www.churchofjesuschrist.org/study/manu...,2025-12-07,2025-12-13,https://www.churchofjesuschrist.org/study/manu...,...,The Articles of Faith and Official Declarations,1 and 2,The Family: A Proclamation to the World,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
print(readings_df['assignment_1'])

50                                             Moroni 10
51                                             Christmas
52     The Restoration of the Fulness of the Gospel o...
53                              Doctrine and Covenants 1
54                           Joseph Smith—History 1:1–26
55                              Doctrine and Covenants 2
56                            Doctrine and Covenants 3–5
57                            Doctrine and Covenants 6–9
58                          Doctrine and Covenants 10–11
59                          Doctrine and Covenants 12–17
60                             Doctrine and Covenants 18
61                             Doctrine and Covenants 19
62                          Doctrine and Covenants 20–22
63                          Doctrine and Covenants 23–26
64                          Doctrine and Covenants 27–28
65                             Doctrine and Covenants 29
66                          Doctrine and Covenants 30–36
67                             

In [ ]:
### Used this cell to quickly scrape most of the titles of books of scripture, then copied the output and saved that as titles. 
### I commented this out because I don't want it to run every time. 

# links = ['https://www.churchofjesuschrist.org/study/scriptures/ot?lang=eng',
#          'https://www.churchofjesuschrist.org/study/scriptures/nt?lang=eng',
#          'https://www.churchofjesuschrist.org/study/scriptures/bofm?lang=eng',
#          'https://www.churchofjesuschrist.org/study/scriptures/dc-testament?lang=eng',
#          'https://www.churchofjesuschrist.org/study/scriptures/pgp?lang=eng']

# titles = []

# for link in links:
#     response = requests.get(link)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     divs = soup.find_all('div',class_='sc-42v4-0 caNosj heading-Vx_DR heading-Vx_DR label')
#     for div in divs:
#         titles.append(div.find('p',class_='title').text)

# # Clean titles using .replace()
# titles = [title.replace('Â\xa0', ' ').strip() for title in titles]


# titles

In [ ]:
titles = ['Genesis','Exodus','Leviticus','Numbers','Deuteronomy',
 'Joshua','Judges','Ruth','1 Samuel','2 Samuel',
 '1 Kings','2 Kings','1 Chronicles','2 Chronicles','Ezra',
 'Nehemiah','Esther','Job','Psalms','Proverbs',
 'Ecclesiastes','Song of Solomon','Isaiah','Jeremiah','Lamentations',
 'Ezekiel','Daniel','Hosea','Joel','Amos',
 'Jonah','Micah','Nahum','Habakkuk','Zephaniah','Haggai',
 'Zechariah','Malachi','Matthew','Mark','Luke',
 'John','Acts','Romans','1 Corinthians','2 Corinthians',
 'Galatians','Ephesians','Philippians','Colossians','1 Thessalonians',
 '2 Thessalonians','1 Timothy','2 Timothy','Titus','Hebrews',
 'James','1 Peter','2 Peter','1 John','Revelation',
 '1 Nephi','2 Nephi','Jacob','Mosiah','Alma',
 'Helaman','3 Nephi','Mormon','Ether','Moroni',
 'Doctrine and Covenants','Official Declarations','Moses','Abraham',
 'Joseph Smith—History','Joseph Smith—Matthew','Articles of Faith']

### 31 Oct 2024

After working out on the other notebooks a better (more accurate) approach to counting the number of lines, I am ready to now get back to working on this calculator. 

I'm thinking that my first course of action needs to be envisioning what I want to end up with. I'm thinking I want to end up with a dataframe populated with dictionaries, populated with dictionaries. I will explain:

* **Dataframe**: The overall dataframe hold as many dictionaries as there are assignment groupings. This will come out to either the number of Sunday schools in a calendar year or the number of Sunday schools plus two to account for Easter and Christmas being their own assignments. Each assignment grouping will be it's own dictionary. 
    * **Assignment grouping dictionaries**: Each assignment grouping dictionary will have the following entries:
        * **assignment dates**: the start and end date for each assignment group
        * **assignment chapters**: a combination of all of the assigned readings within the assignment group period
        * **daily breakdown dictionary**: a dictionary containing the following information for each date within the assignment group dates
            * **weekday**: the day of the week
            * **date**: the date of the reading assignment
            * **assignment**: the reading assignment for the day

By far the most difficult part of this process will be breaking down the daily assignment. However, this set up is also the setup I'll be using when I create my crazy loops and iterations. 

In [ ]:
readings_df_copy = readings_df

In [ ]:
readings_df_copy

,reading_for_sunday,sunday_number,sunday_class,assignment,start_date,end_date,link,reading_group,assignment_1,assignment_2,assignment_1_book,assignment_1_chapters,assignment_2_book,assignment_2_chapters
50,2024-12-22,4,EQ/RS/YM/YW,Moroni 10,2024-12-15,2024-12-21,https://www.churchofjesuschrist.org/study/manu...,23,Moroni 10,None,Moroni,10,<NA>,<NA>
51,2024-12-29,5,Fifth Sunday,Christmas,2024-12-22,2024-12-28,https://www.churchofjesuschrist.org/study/manu...,23,Christmas,None,Christmas,<NA>,<NA>,<NA>
52,2025-01-05,1,Sunday School,The Restoration of the Fulness of the Gospel o...,2024-12-29,2025-01-04,https://www.churchofjesuschrist.org/study/manu...,23,The Restoration of the Fulness of the Gospel o...,None,The Restoration of the Fulness of the Gospel o...,<NA>,<NA>,<NA>
53,2025-01-12,2,EQ/RS/YM/YW,Doctrine and Covenants 1,2025-01-05,2025-01-11,https://www.churchofjesuschrist.org/study/manu...,24,Doctrine and Covenants 1,None,Doctrine and Covenants,1,<NA>,<NA>
54,2025-01-19,3,Sunday School,Joseph Smith—History 1:1–26,2025-01-12,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,24,Joseph Smith—History 1:1–26,None,Joseph Smith—History,1:1–26,<NA>,<NA>
55,2025-01-26,4,EQ/RS/YM/YW,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-01-25,https://www.churchofjesuschrist.org/study/manu...,25,Doctrine and Covenants 2,Joseph Smith—History 1:27–65,Doctrine and Covenants,2,Joseph Smith—History,1:27–65
56,2025-02-02,1,Sunday School,Doctrine and Covenants 3–5,2025-01-26,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,25,Doctrine and Covenants 3–5,None,Doctrine and Covenants,3–5,<NA>,<NA>
57,2025-02-09,2,EQ/RS/YM/YW,Doctrine and Covenants 6–9,2025-02-02,2025-02-08,https://www.churchofjesuschrist.org/study/manu...,26,Doctrine and Covenants 6–9,None,Doctrine and Covenants,6–9,<NA>,<NA>
58,2025-02-16,3,Sunday School,Doctrine and Covenants 10–11,2025-02-09,2025-02-15,https://www.churchofjesuschrist.org/study/manu...,26,Doctrine and Covenants 10–11,None,Doctrine and Covenants,10–11,<NA>,<NA>
59,2025-02-23,4,EQ/RS/YM/YW,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-02-22,https://www.churchofjesuschrist.org/study/manu...,27,Doctrine and Covenants 12–17,Joseph Smith—History 1:66–75,Doctrine and Covenants,12–17,Joseph Smith—History,1:66–75


### 1 Nov 2024

So the first step of my process will be to get the **assignment grouping dictionaries** made. Once those are made I can populate them with the information I  need.

Before doing that, though, I want to correct the reading group column. 

In [ ]:
### Commented out because I inserted it above in a place that makes more sense

# readings_df = readings_df_copy

# readings_df['reading_group'] = readings_df['reading_group'] - (readings_df['reading_group'].min()-1)

# readings_df

,reading_for_sunday,sunday_number,sunday_class,assignment,start_date,end_date,link,reading_group,assignment_1,assignment_2,assignment_1_book,assignment_1_chapters,assignment_2_book,assignment_2_chapters
50,2024-12-22,4,EQ/RS/YM/YW,Moroni 10,2024-12-15,2024-12-21,https://www.churchofjesuschrist.org/study/manu...,1,Moroni 10,None,Moroni,10,<NA>,<NA>
51,2024-12-29,5,Fifth Sunday,Christmas,2024-12-22,2024-12-28,https://www.churchofjesuschrist.org/study/manu...,1,Christmas,None,Christmas,<NA>,<NA>,<NA>
52,2025-01-05,1,Sunday School,The Restoration of the Fulness of the Gospel o...,2024-12-29,2025-01-04,https://www.churchofjesuschrist.org/study/manu...,1,The Restoration of the Fulness of the Gospel o...,None,The Restoration of the Fulness of the Gospel o...,<NA>,<NA>,<NA>
53,2025-01-12,2,EQ/RS/YM/YW,Doctrine and Covenants 1,2025-01-05,2025-01-11,https://www.churchofjesuschrist.org/study/manu...,2,Doctrine and Covenants 1,None,Doctrine and Covenants,1,<NA>,<NA>
54,2025-01-19,3,Sunday School,Joseph Smith—History 1:1–26,2025-01-12,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,2,Joseph Smith—History 1:1–26,None,Joseph Smith—History,1:1–26,<NA>,<NA>
55,2025-01-26,4,EQ/RS/YM/YW,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-01-25,https://www.churchofjesuschrist.org/study/manu...,3,Doctrine and Covenants 2,Joseph Smith—History 1:27–65,Doctrine and Covenants,2,Joseph Smith—History,1:27–65
56,2025-02-02,1,Sunday School,Doctrine and Covenants 3–5,2025-01-26,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,3,Doctrine and Covenants 3–5,None,Doctrine and Covenants,3–5,<NA>,<NA>
57,2025-02-09,2,EQ/RS/YM/YW,Doctrine and Covenants 6–9,2025-02-02,2025-02-08,https://www.churchofjesuschrist.org/study/manu...,4,Doctrine and Covenants 6–9,None,Doctrine and Covenants,6–9,<NA>,<NA>
58,2025-02-16,3,Sunday School,Doctrine and Covenants 10–11,2025-02-09,2025-02-15,https://www.churchofjesuschrist.org/study/manu...,4,Doctrine and Covenants 10–11,None,Doctrine and Covenants,10–11,<NA>,<NA>
59,2025-02-23,4,EQ/RS/YM/YW,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-02-22,https://www.churchofjesuschrist.org/study/manu...,5,Doctrine and Covenants 12–17,Joseph Smith—History 1:66–75,Doctrine and Covenants,12–17,Joseph Smith—History,1:66–75


That should make it easier to work with the reading groups in the future.

I also realize that I need to update the 'reading_for_sunday' column to reflect the final date of reading, not just every Sunday of the year. It will also be good to establish a start and end date for each reading group. This might be most easily accomplished by creating a subset of the dataframe for each group. 

In [ ]:
# Commented out because I added it above, in a place where it makes more sense

# subsets_list = []

# for i in range(1, readings_df['reading_group'].max()+1):
#     subset_df = readings_df[readings_df['reading_group'] == i]
#     subset_df['group_start_date'] = subset_df['start_date'].min()
#     subset_df['group_end_date'] = subset_df['end_date'].max()
#     subset_df['reading_for_sunday'] = subset_df['group_end_date'] + timedelta(days=1)
#     subsets_list.append(subset_df)

# readings_df = pd.concat(subsets_list, ignore_index=True)

C:\Users\bfran\AppData\Local\Temp\ipykernel_5128\3533641557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['group_start_date'] = subset_df['start_date'].min()
C:\Users\bfran\AppData\Local\Temp\ipykernel_5128\3533641557.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['group_end_date'] = subset_df['end_date'].max()
C:\Users\bfran\AppData\Local\Temp\ipykernel_5128\3533641557.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [ ]:
readings_df_1

,reading_for_sunday,sunday_number,sunday_class,assignment,start_date,end_date,link,reading_group,assignment_1,assignment_2,assignment_1_book,assignment_1_chapters,assignment_2_book,assignment_2_chapters,group_start_date,group_end_date
0,2025-01-05,4,EQ/RS/YM/YW,Moroni 10,2024-12-15,2024-12-21,https://www.churchofjesuschrist.org/study/manu...,1,Moroni 10,None,Moroni,10,NaN,NaN,2024-12-15,2025-01-04
1,2025-01-05,5,Fifth Sunday,Christmas,2024-12-22,2024-12-28,https://www.churchofjesuschrist.org/study/manu...,1,Christmas,None,Christmas,<NA>,NaN,NaN,2024-12-15,2025-01-04
2,2025-01-05,1,Sunday School,The Restoration of the Fulness of the Gospel o...,2024-12-29,2025-01-04,https://www.churchofjesuschrist.org/study/manu...,1,The Restoration of the Fulness of the Gospel o...,None,The Restoration of the Fulness of the Gospel o...,<NA>,NaN,NaN,2024-12-15,2025-01-04
3,2025-01-19,2,EQ/RS/YM/YW,Doctrine and Covenants 1,2025-01-05,2025-01-11,https://www.churchofjesuschrist.org/study/manu...,2,Doctrine and Covenants 1,None,Doctrine and Covenants,1,NaN,NaN,2025-01-05,2025-01-18
4,2025-01-19,3,Sunday School,Joseph Smith—History 1:1–26,2025-01-12,2025-01-18,https://www.churchofjesuschrist.org/study/manu...,2,Joseph Smith—History 1:1–26,None,Joseph Smith—History,1:1–26,NaN,NaN,2025-01-05,2025-01-18
5,2025-02-02,4,EQ/RS/YM/YW,Doctrine and Covenants 2; Joseph Smith—History...,2025-01-19,2025-01-25,https://www.churchofjesuschrist.org/study/manu...,3,Doctrine and Covenants 2,Joseph Smith—History 1:27–65,Doctrine and Covenants,2,Joseph Smith—History,1:27–65,2025-01-19,2025-02-01
6,2025-02-02,1,Sunday School,Doctrine and Covenants 3–5,2025-01-26,2025-02-01,https://www.churchofjesuschrist.org/study/manu...,3,Doctrine and Covenants 3–5,None,Doctrine and Covenants,3–5,<NA>,<NA>,2025-01-19,2025-02-01
7,2025-02-16,2,EQ/RS/YM/YW,Doctrine and Covenants 6–9,2025-02-02,2025-02-08,https://www.churchofjesuschrist.org/study/manu...,4,Doctrine and Covenants 6–9,None,Doctrine and Covenants,6–9,NaN,NaN,2025-02-02,2025-02-15
8,2025-02-16,3,Sunday School,Doctrine and Covenants 10–11,2025-02-09,2025-02-15,https://www.churchofjesuschrist.org/study/manu...,4,Doctrine and Covenants 10–11,None,Doctrine and Covenants,10–11,NaN,NaN,2025-02-02,2025-02-15
9,2025-03-02,4,EQ/RS/YM/YW,Doctrine and Covenants 12–17; Joseph Smith—His...,2025-02-16,2025-02-22,https://www.churchofjesuschrist.org/study/manu...,5,Doctrine and Covenants 12–17,Joseph Smith—History 1:66–75,Doctrine and Covenants,12–17,Joseph Smith—History,1:66–75,2025-02-16,2025-03-01
